In [1]:
# Aditya Tripathi, Anurag Choudhury, Karan Sharma
# 2023 180 46, 2023 180 59, 2023 180 18
# MSc DS (Sem2)
import json
import socket
import threading
import hashlib
from time import time, sleep

In [2]:
class BlockchainNode:
    def __init__(self, host, port):
        self.host = host
        self.port = port
        self.peers = set()
        self.chain = []

        self.server_thread = threading.Thread(target=self.run_server, daemon=True)
        self.server_thread.start()

    def run_server(self):
        server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        server.bind((self.host, self.port))
        server.listen(5)

        print(f"Node listening on {self.host}:{self.port}")

        while True:
            client_socket, addr = server.accept()
            client_thread = threading.Thread(target=self.handle_client, args=(client_socket,))
            client_thread.start()

    def handle_client(self, client_socket):
        data = client_socket.recv(1024).decode('utf-8')

        try:
            message = json.loads(data)
            self.process_message(message)
        except json.JSONDecodeError:
            print("Invalid JSON received.")

        client_socket.close()

    def process_message(self, message):
        if 'type' not in message or 'data' not in message:
            print("Invalid message format.")
            return

        message_type = message['type']
        message_data = message['data']

        if message_type == 'connect':
            self.connect_to_peer(message_data)
        elif message_type == 'update_chain':
            self.update_chain(message_data)
        elif message_type == 'get_chain':
            self.send_chain()
        elif message_type == 'mine':
            self.mine_block(message_data)

    def connect_to_peer(self, peer_address):
        if peer_address not in self.peers and peer_address != (self.host, self.port):
            print(f"Connecting to {peer_address}")
            self.peers.add(peer_address)

            self.send_chain(peer_address)

            self.broadcast({'type': 'connect', 'data': peer_address})

    def update_chain(self, new_chain):
        if len(new_chain) > len(self.chain):
            print("Updating chain.")
            self.chain = new_chain
            self.broadcast({'type': 'update_chain', 'data': self.chain})

    def send_chain(self, target_address=None):
        message = {'type': 'get_chain', 'data': self.chain}

        if target_address:
            self.send_message(target_address, message)
        else:
            for peer in self.peers:
                self.send_message(peer, message)

    def send_message(self, target_address, message):
        try:
            client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            client_socket.connect(target_address)
            client_socket.send(json.dumps(message).encode('utf-8'))
            client_socket.close()
        except ConnectionRefusedError:
            print(f"Could not connect to {target_address}")

    def mine_block(self, transactions):
        last_block = self.chain[-1]
        proof = 0

        block = {
            'index': len(self.chain) + 1,
            'timestamp': time(),
            'transactions': transactions,
            'proof': proof,
            'previous_hash': self.hash(last_block),
        }

        self.chain.append(block)
        self.broadcast({'type': 'update_chain', 'data': self.chain})

    def hash(self, block):
        block_string = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()

    def broadcast(self, message):
        for peer in self.peers:
            self.send_message(peer, message)

In [ ]:
node1 = BlockchainNode('10.200.10.67', 5000)
node2 = BlockchainNode('localhost', 5001)

node1.connect_to_peer(('localhost', 5001))
node2.connect_to_peer(('10.200.10.67', 5000))

sleep(2)
node1.send_chain()
node2.send_chain()

transactions_node1 = [{'sender': 'Alice', 'recipient': 'Bob', 'amount': 10}]
node1.mine_block(transactions_node1)

transactions_node2 = [{'sender': 'Bob', 'recipient': 'Charlie', 'amount': 5}]
node2.mine_block(transactions_node2)

while True:
    pass